## MCP + LangGraph 튜토리얼

reference: https://github.com/teddynote-lab/langgraph-mcp-agents/blob/master/MCP-HandsOn-KOR.ipynb


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### MultiServerMCPClient

사전에 `mcp_server_local.py`를 싱행해둡니다. 터미널을 열고 가상환경이 활성화 되어있는 상태에서 서버를 실행해 주세요.  
`async with`로 일시적인 Session 연결을 생성 후 해제

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

  + Exception Group Traceback (most recent call last):
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_16695/1333718561.py", line 8, in <module>
  |     async with MultiServerMCPClient(
  |                ^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 266, in __aenter__
  |     await self.connect_to_server_via_sse(server_name, **connection_dict)
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 232, in connect_to_server_via_sse
  |     sse_transport = await self.exit_stack.enter_async_context(
  |                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/contextlib.py", line 659, in e

In [3]:
answer

NameError: name 'answer' is not defined

In [4]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. 명시적으로 연결 초기화(이 부분이 필요함)
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())

  + Exception Group Traceback (most recent call last):
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_16695/2954019773.py", line 12, in <module>
  |     await client.__aenter__()
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 266, in __aenter__
  |     await self.connect_to_server_via_sse(server_name, **connection_dict)
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/site-packages/langchain_mcp_adapters/client.py", line 232, in connect_to_server_via_sse
  |     sse_transport = await self.exit_stack.enter_async_context(
  |                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/taejong_kim/anaconda3/envs/mcp/lib/python3.12/contextlib.py", line 659, in enter_async_context
  |     result = await _ente

In [5]:
agent = create_react_agent(model, client.get_tools())

await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 서울의 날씨 정보를 실시간으로 확인하기 위해서는 기상청 날씨누리(www.weather.go.kr) 또는 다른 날씨 앱을 확인하는 것이 가장 정확합니다.  제가 실시간 정보를 제공할 수는 없기 때문입니다.

하지만 일반적으로 서울의 날씨는 다음과 같은 특징을 가지고 있습니다.

* **봄 (3월~5월):** 일교차가 크고, 황사나 미세먼지의 영향을 받는 날이 있습니다. 따뜻하고 맑은 날이 많아지지만, 변덕스러운 날씨 변화를 보이기도 합니다.
* **여름 (6월~8월):** 고온다습하며 장마철이 있습니다. 집중호우가 잦고, 무더위가 지속됩니다.
* **가을 (9월~11월):** 맑고 건조한 날이 많으며, 일교차가 큽니다. 단풍이 아름답게 물드는 계절입니다.
* **겨울 (12월~2월):** 춥고 건조하며, 눈이 내리는 날이 있습니다. 한파가 몰아치는 경우도 있습니다.


현재 시기의 서울 날씨를 알고 싶다면, 위에 언급된 날씨 정보 사이트나 앱을 확인해주세요.

{'node': 'agent',
 'content': AIMessageChunk(content=' 한파가 몰아치는 경우도 있습니다.\n\n\n현재 시기의 서울 날씨를 알고 싶다면, 위에 언급된 날씨 정보 사이트나 앱을 확인해주세요.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-5fa34a2d-9d9a-4263-96b4-382c37e75546', usage_metadata={'input_tokens': -1, 'output_tokens': 311, 'total_tokens': 310, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'langgraph_step': 1,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:1eb68bba-4e06-01d1-44f1-42e2b422f15f',
  'checkpoint_ns': 'agent:1eb68bba-4e06-01d1-44f1-42e2b422f15f',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

### Stdio 통신 방식

- Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.
- 통신을 위해 표준 입력/출력 사용

아래 python 경로는 수정

In [2]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import astream_graph

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

server_params = StdioServerParameters(
    command="/usr/local/anaconda3/envs/mcp-agent/bin/python",
    args=["mcp_server_local.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)

        # 에이전트 생성
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

I0000 00:00:1743511884.781788  200678 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x115110220>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울!
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - 

## RAG를 구축한 MCP 서버 사용

- file: `mcp_server_rag.py`
- stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever`도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py`에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 않아도 됩니다.

In [4]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_teddynote.messages import astream_graph
from dotenv import load_dotenv
load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

server_params = StdioServerParameters(
    command="/usr/local/anaconda3/envs/mcp-agent/bin/python",
    args=["./mcp_server_rag.py"]
)

# Stdio 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()

        tools = await load_mcp_tools(session)

        agent = create_react_agent(model, tools)

        await astream_graph(
            agent, {"messages": "DeepSeek-R1 논문의 Abstract를 찾아줘"}
        )

I0000 00:00:1743511946.333773  200678 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers



🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
3.1. DeepSeek-R1 Evaluation
Benchmark (Metric)
Claude-3.5- GPT-4o DeepSeek OpenAI OpenAI DeepSeek
Sonnet-1022
0513
V3
o1-mini o1-1217
R1
Architecture
-
-
MoE
-
-
MoE
# Activated Params
-
-
37B
-
-
37B
# Total Params
-
-
671B
-
-
671B
English
MMLU (Pass@1)
88.3
87.2
88.5
85.2
91.8
90.8
MMLU-Redux (EM)
88.9
88.0
89.1
86.7
-
92.9
MMLU-Pro (EM)
78.0
72.6
75.9
80.3
-
84.0
DROP (3-shot F1)
88.3
83.7
91.6
83.9
90.2
92.2
IF-Eval (Prompt Strict)
86.5
84.3
86.1
84.8
-
83.3
GPQA Diamond (Pass@1)
65.0
49.9
59.1
60.0
75.7
71.5
SimpleQA (Correct)
28.4
38.2
24.9
7.0
47.0
30.1
FRAMES (Acc.)
72.5
80.5
73.3
76.9
-
82.5
AlpacaEval2.0 (LC-winrate)
52.0
51.1
70.0
57.8
-
87.6
ArenaHard (GPT-4-1106)
85.2
80.4
85.5
92.0
-
92.3
Code
LiveCodeBench (Pass@1-COT)
38.9
32.9
36.2
53.8
63.4
65.9
Codeforces (Percentile)
20.3
23.6
58.7
93.4
96.6
96.3
Codeforces (Rating)
717
759
1134
18

### SSE 방식과 Stdio 방식 혼합 사용
- 파일: `mcp_server_rag.py`는 Stdio 방식으로 통신
- `langchain-dev-docs`는 SSE 방식으로 통신
  

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "/home/taejong_kim/anaconda3/envs/mcp/bin/python",  # conda env 경로
            "args": ["./mcp_server_rag.py"],
            "transport": 'stdio'
        },
        "langchain-dev-docs": {
            "url": "http://teddynote.io:8765/sse",
            "transport": "sse",
        }
    }
)

await client.__aenter__()

In [33]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in Korean."
)

agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

In [22]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
# await astream_graph(
#     agent,
#     {
#         "messages": "`retriever` 도구를 사용해서 딥시크 R1 논문을 요약해줘",
#     },
#     config=config,
# )

In [5]:
# config = RunnableConfig(recursion_limit=30, thread_id=1)
# await astream_graph(
#     agent,
#     {"messages": "langgraph-dev-docs 참고해서 self-rag의 정의에 대해서 알려줘"},
#     config=config,
# )

In [6]:
# await astream_graph(
#     agent, {"messages": "이전의 내용을 bullet point 로 요약해줘"}, config=config
# )

### Langchain에 통합된 도구 + MCP 도구

여기서는 LangChain에 통합된 도구를 기존의 MCP로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [7]:
from langchain_teddynote.tools.tavily import TavilySearch

tavily = TavilySearch(max_results=3, topic="news", days=3)

# 기존의 MCP도구와 함께 사용합니다.
tools = client.get_tools() + [tavily]

In [8]:

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in Korean."
agent = create_react_agent(
    model, tools, prompt=prompt, checkpointer=MemorySaver())

In [49]:
await astream_graph(agent, {"messages": "오늘 AI관련 뉴스 찾아줘"}, config=config)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://www.pymnts.com/artificial-intelligence-2/2025/apple-reportedly-developing-ai-agent-doctors-in-latest-health-push/", "raw_content": "Published Time: 2025-03-30T23:18:34+00:00\nApple Reportedly Developing AI Agent 'Doctors'\nApple Reportedly Developing AI Agent 'Doctors'\n===============\nYour browser does not support the video tag.\nWATCH NOW | SUBSCRIBE\nSearch\nPYMNTS TV\nToday\nB2B\nRetail\nFintech\nDigital Transformation\nCrypto\nAI\nPYMNTS® Intelligence\nTrackers\nProprietary Data Studies\nPYMNTS Data Lab\nPYMNTS® Intelligence\nTrackers\nProprietary Data Studies\nPYMNTS Data Lab\nMarkets\nEvents\nMore\nTopics\nArtifical Intelligence\nConnected Car\nBuy Now Pay Later\nBanking\nCloud\nCross-Border Payments\nGig-Economy\nGrocery & Pharmacy\nHealthcare Payments\nInsurtech\nSmall & Medium Businesses\nSocial Platforms\nSubscription Comm

{'node': 'agent',
 'content': AIMessageChunk(content='과 무역 관세에 대한 우려로 하락했습니다. (TipRanks)\n\n\n더 자세한 내용을 원하시면 위에 제공된 링크를 참조하시거나, 다른 키워드로 다시 검색해 보세요.', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-41b58fe4-6e4c-4360-9bc9-3dee760f4b00', usage_metadata={'input_tokens': -710, 'output_tokens': 251, 'total_tokens': -459, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 6,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:7efc0b51-b887-3590-6e90-fc29cf9d35f7',
  'checkpoint_ns': 'agent:7efc0b51-b887-3590-6e90-fc29cf9d35f7',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

### Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/
- 사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화
참고

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 "transport": "stdio" 로 꼭 설정해야 합니다.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
load_dotenv()  # 노트북 시작 부분에 추가


model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "document-retriever": {
            "command": "/usr/local/anaconda3/envs/mcp-agent/bin/python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
    }
)

await client.__aenter__()

I0000 00:00:1743512670.523907  200678 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1743512699.624086  200678 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1743512704.701413  200678 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [18]:

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [19]:
client.get_tools()

[StructuredTool(name='sequentialthinking', description='A detailed tool for dynamic and reflective problem-solving through thoughts.\nThis tool helps analyze problems through a flexible thinking process that can adapt and evolve.\nEach thought can build on, question, or revise previous insights as understanding deepens.\n\nWhen to use this tool:\n- Breaking down complex problems into steps\n- Planning and design with room for revision\n- Analysis that might need course correction\n- Problems where the full scope might not be clear initially\n- Problems that require a multi-step solution\n- Tasks that need to maintain context over multiple steps\n- Situations where irrelevant information needs to be filtered out\n\nKey features:\n- You can adjust total_thoughts up or down as you progress\n- You can question or revise previous thoughts\n- You can add more thoughts even after reaching what seemed like the end\n- You can express uncertainty and explore alternative approaches\n- Not every t

In [20]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

agent = create_react_agent(model, client.get_tools(),
                           checkpointer=MemorySaver())

In [26]:
from langchain_teddynote.messages import astream_graph

config = RunnableConfig(recursion_limit=30, thread_id=1)

await astream_graph(
    agent,
    {
        "messages": "현재 경로를 포함한 하위 폴더 구조를 tree로 그려줘. 단 .venv 폴더와 .git 폴더는 제외하고 출력해줘"
    },
    config=config,
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[FILE] .env
[DIR] .git
[FILE] .gitignore
[FILE] MCP-HandsOn.ipynb
[FILE] README.md
[DIR] data
[FILE] mcp_server_local.py
[FILE] mcp_server_rag.py

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[FILE] DeepSeek-R1.pdf

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
.
├── .env
├── .gitignore
├── MCP-HandsOn.ipynb├── README.md
├── data
│   └── DeepSeek-R1.pdf
├── mcp_server_local.py
└── mcp_server_rag.py

{'node': 'agent',
 'content': AIMessageChunk(content='.pdf\n├── mcp_server_local.py\n└── mcp_server_rag.py', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-7388916d-dfdd-4e1c-b64a-eaa2edf373c0', usage_metadata={'input_tokens': -1032, 'output_tokens': 59, 'total_tokens': -973, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 28,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:f25ba4c7-89cc-05d7-7c9c-20147a80a794',
  'checkpoint_ns': 'agent:f25ba4c7-89cc-05d7-7c9c-20147a80a794',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}

In [25]:
await astream_graph(
    agent,
    {
        "messages": (
            "`retriever` 도구를 사용해서 DeepSeek-R1의 모델 학습 방법 관련 내용을 검색하고"
            "`Sequential Thinking` 도구를 사용해서 보고서를 작성해줘"
        )
    },
    config=config,
)

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 

🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via
Reinforcement Learning
DeepSeek-AI
research@deepseek.com
Abstract
We introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.
DeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-
vised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.
Through RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing
reasoning behaviors. However, it encounters challenges such as poor readability, and language
mixing. To address these issues and further enhance reasoning performance, we introduce
DeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-
R1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the
research commun

Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is not supported in schema, ignoring
Key 'additionalProperties' is not supported in schema, ignoring
Key '$schema' is


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
## DeepSeek-R1 모델 학습 방법 보고서

**서론:**DeepSeek-R1은 DeepSeek에서 개발한 추론 능력이 뛰어난 대규모 언어 모델입니다. 이 보고서는 DeepSeek-R1의 학습 방법에 대해 자세히 설명합니다.

**DeepSeek-R1-Zero:**

DeepSeek-R1-Zero는 지도 미세 조정(SFT) 없이 대규모 강화 학습(RL)을 통해 훈련된 모델입니다. 이 모델은 놀라운 추론 능력을 보여주지만, 가독성이 떨어지고 언어 혼합 문제가 발생하는 등의 단점이 있습니다.

**DeepSeek-R1:**

DeepSeek-R1은 DeepSeek-R1-Zero의 단점을 해결하고 추론 성능을 더욱 향상시키기 위해 개발되었습니다. DeepSeek-R1은 RL을 사용하기 전에 다단계 훈련 및 cold-start 데이터를 활용합니다.

**학습 방법:**

1. **Cold-Start:** RL 훈련의 초기 불안정한 cold-start 단계를 방지하기 위해 DeepSeek-R1은 소량의 long CoT 데이터를 수집하고 모델을 미세 조정합니다.  long CoT 데이터는 few-shot 프롬프팅, 상세 답변 생성 프롬프팅, DeepSeek-R1-Zero 출력 재구성, 사람의 후처리 등의 방법으로 수집됩니다.

2. **강화 학습 (RL):** DeepSeek-R1은 강화 학습을 통해 추론 능력을 향상시킵니다. 특히 코딩, 수학, 과학 및 논리 추론과 같은 추론 집약적인 작업에 중점을 둡니다. RL 훈련 중 언어 혼합 문제를 완화하기 위해 언어 일관성 보상을 도입합니다. 이 보상은 CoT에서 대상 언어 단어의 비율로 계산됩니다.

3. **Distillation:** DeepSeek-R1은 추론 능력을 소형 모델로 증류하는 데 사용됩니다. DeepSeek-R1을 교사 모델로 사용하여 800K 훈련 샘플을 생성하고 여러 소형 모델

{'node': 'agent',
 'content': AIMessageChunk(content=' 작업에 대한 성능 향상', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-3dedfaca-6a01-40af-9436-2a6d9bcbb45d', usage_metadata={'input_tokens': -983, 'output_tokens': 662, 'total_tokens': -321, 'input_token_details': {'cache_read': 0}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 21,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:f7dc9f01-ee17-8b85-d5b8-688a06c9037d',
  'checkpoint_ns': 'agent:f7dc9f01-ee17-8b85-d5b8-688a06c9037d',
  'ls_provider': 'google_genai',
  'ls_model_name': 'models/gemini-1.5-pro',
  'ls_model_type': 'chat',
  'ls_temperature': 0.7}}